In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as itk
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage.filters import gaussian_filter1d
from scipy.ndimage import convolve
from skimage.transform import rescale, resize, downscale_local_mean
sys.path.append('/opt/medana')
import cv2

from amslib import resample_image

In [ ]:
#funkcije
def img2array(img):
    return np.squeeze(itk.GetArrayFromImage(img))

def array2img(array):
    return np.squeeze(itk.GetImageFromArray(array))

def slicer(img):
    #print('Velikost slike je {}'.format(img.GetSize()))
    velikost = img.GetSize()
    rezina_x = img[int(velikost[0]/2),:,:]
    rezina_y = img[:,int(velikost[1]/2),:]
    rezina_z = img[:,:,int(velikost[2]/2)]
    rezine = [rezina_x, rezina_y, rezina_z]
    return rezine

def extract_image(image, output_size, interpolation_type=itk.sitkLinear):
        input_size = image.GetSize()
        print('Originalna velikost: ', velikost)
        if velikost[0] > velikost[2] and velikost[1] > velikost[2]:
            new_spacing_mm = ( input_size[0] / output_size[0], input_size[1] / output_size[1], input_size[2] / output_size[2])
            #new_spacing_mm = ( 2, 2, 2)
            resampled =resample_image(
            itk.RegionOfInterest(image, (velikost[0], velikost[1], velikost[2], 1)), 
            spacing_mm = new_spacing_mm, 
            inter_type=interpolation_type)
        elif velikost[0] > velikost[1] and velikost[2] > velikost[1]:
            new_spacing_mm = ( input_size[0] / output_size[0], input_size[1] / output_size[1], input_size[2] / output_size[2])
            #new_spacing_mm = ( 2, 2, 2)
            resampled = resample_image(
            itk.RegionOfInterest(image, (velikost[0], velikost[1], velikost[2], 1)), 
            spacing_mm = new_spacing_mm, 
            inter_type=interpolation_type)
        elif velikost[1] > velikost[0] and velikost[2] > velikost[0]:
            new_spacing_mm = ( input_size[0] / output_size[0], input_size[1] / output_size[1], input_size[2] / output_size[2])
            #new_spacing_mm = ( 2, 2, 2)
            resampled = resample_image(
            itk.RegionOfInterest(image, (velikost[0], velikost[1], velikost[2], 1)), 
            spacing_mm = new_spacing_mm, 
            inter_type=interpolation_type)
        return resampled

In [ ]:
# PRIPRAVA SLIK IZ IXI
from tqdm import tqdm
from os.path import join
# naloži vse podatke in obreži slike na velikost deljivo z 2
mri_data_testna = []

DATA_PATH = './ixi_data/t1-images'
OUTPUT_DIR = './ixi_data/t1-images-resampled32'
OUTPUT_SIZE = (32,32,32)

patient_paths = os.listdir(DATA_PATH)
print('V mapi {:s} je {:d} podmap.'.format(DATA_PATH, len(patient_paths)))
i = 0
for files in os.listdir(DATA_PATH):
    if files.startswith("IXI"):
        t1 = itk.ReadImage(os.path.join(DATA_PATH, files))
        velikost = t1.GetSize()
        print('Originalna velikost: ', velikost)
        if velikost[0] > velikost[2] and velikost[1] > velikost[2]:
            #img_float = itk.Cast(t1, itk.sitkFloat32)
            #img = img2array(img_float)
            #t1_g = gaussian_filter(img, sigma=(2,2,1))
            t1_g = extract_image(t1, OUTPUT_SIZE, itk.sitkNearestNeighbor)
            t1_g = img2array(t1_g)
            t1_g = resize(t1_g, OUTPUT_SIZE)
            t1_out = itk.GetImageFromArray(t1_g)
            velikosto = t1_out.GetSize()
            #velikosto = t1_g.GetSize()
            print('Decimirana velikost: ', velikosto)
        elif velikost[0] > velikost[1] and velikost[2] > velikost[1]:
            #img_float = itk.Cast(t1, itk.sitkFloat32)
            #img = img2array(img_float)
            #t1_g = gaussian_filter(img, sigma=(2,2,1))
            t1_g = extract_image(t1, OUTPUT_SIZE, itk.sitkNearestNeighbor)
            t1_g = img2array(t1_g)
            t1_g = resize(t1_g, OUTPUT_SIZE)
            t1_out = itk.GetImageFromArray(t1_g)
            velikosto = t1_out.GetSize()
            #velikosto = t1_g.GetSize()
            print('Decimirana velikost: ', velikosto)
        elif velikost[1] > velikost[0] and velikost[2] > velikost[0]:
            #img_float = itk.Cast(t1, itk.sitkFloat32)
            #img = img2array(img_float)
            #t1_g = gaussian_filter(img, sigma=(2,2,1))
            t1_g = extract_image(t1, OUTPUT_SIZE, itk.sitkNearestNeighbor)
            t1_g = img2array(t1_g)
            t1_g = resize(t1_g, OUTPUT_SIZE)
            t1_out = itk.GetImageFromArray(t1_g)
            velikosto = t1_out.GetSize()
            #velikosto = t1_g.GetSize()
            print('Decimirana velikost: ', velikosto)
            
        i = i + 1
        print('OK',i, files)
        #itk.WriteImage(t1_out, os.path.join(OUTPUT_DIR, files))
        
        rezine_obg = slicer(t1_out)
        #prikaz
        f, (ax1, ax2, ax3) = plt.subplots(1, 3)
        ax1.imshow(img2array(rezine_obg[0]), cmap='gray')
        ax1.set_title('Rezina X - podvz-g')
        ax1.axis('off')
        ax2.imshow(img2array(rezine_obg[1]), cmap='gray')
        ax2.set_title('Rezina y - podvz-g')
        ax2.axis('off')
        ax3.imshow(img2array(rezine_obg[2]), cmap='gray')
        ax3.set_title('Rezina Z - podvz-g')
        ax3.axis('off')
        plt.show()

In [ ]:
#ustvarim folderje za vsako kategorijo
t0_pth = './ixi_data/t1-images-or'
orientacije = ['AIL', 'AIR', 'ALI', 'ALS', 'ARI', 'ARS', 'ASL', 'ASR',
               'IAL', 'IAR', 'ILA', 'ILP', 'IPL', 'IPR', 'IRA', 'IRP',
               'LAI', 'LAS', 'LIA', 'LIP', 'LPI', 'LPS', 'LSA', 'LSP',
               'PIL', 'PIR', 'PLI', 'PLS', 'PRI', 'PRS', 'PSL', 'PSR',
               'RAI', 'RAS', 'RIA', 'RIP', 'RPI', 'RPS', 'RSA', 'RSP',
               'SAL', 'SAR', 'SLA', 'SLP', 'SPL', 'SPR', 'SRA', 'SRP']
i = 0
for i in orientacije:
    os.mkdir(t0_pth + '/' +i)
    


In [ ]:
#reorientacija vsake slike v eno izmed 48ih testov
t1_pth = './ixi_data/t1-images-resampled32'
OUTPUT_DIR = './ixi_data/t1-images-rsori32'
orientacije = ['AIL', 'AIR', 'ALI', 'ALS', 'ARI', 'ARS', 'ASL', 'ASR',
               'IAL', 'IAR', 'ILA', 'ILP', 'IPL', 'IPR', 'IRA', 'IRP',
               'LAI', 'LAS', 'LIA', 'LIP', 'LPI', 'LPS', 'LSA', 'LSP',
               'PIL', 'PIR', 'PLI', 'PLS', 'PRI', 'PRS', 'PSL', 'PSR',
               'RAI', 'RAS', 'RIA', 'RIP', 'RPI', 'RPS', 'RSA', 'RSP',
               'SAL', 'SAR', 'SLA', 'SLP', 'SPL', 'SPR', 'SRA', 'SRP']
i = 0
l = 1
tabela = []

# hkrati podatke pišem tudi v csv file
with open('./ixi_data/t1-images-rsori32/traindata.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for files in os.listdir(t1_pth):
        if files.startswith("IXI1"):
            t1 = itk.ReadImage(os.path.join(t1_pth, files))
            velikost = t1.GetSize()
            ori = OrientImage()
            for i in orientacije:
                #print(i)
                ori.orientation = i
                print('OK',i,files)
                ori.run(t1)
                t1_ori = ori.get_output()
                itk.WriteImage(t1_ori, os.path.join(OUTPUT_DIR + '/' + i, i + str(l) + '.nii.gz'))
                newfiles = os.path.join(OUTPUT_DIR + '/' + i, i + str(l) + '.nii.gz')
                filewriter.writerow([newfiles, i])
                tabela = tabela + [(l, i, newfiles)]
                l = l+1
print(tabela)

rezine = slicer(t1)
rezine_o = slicer(t1_ori)
#prikaz samo za referenco
f, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(img2array(rezine[0]), cmap='gray')
ax1.set_title('Rezina X - original')
ax1.axis('off')
ax2.imshow(img2array(rezine_o[0]), cmap='gray')
ax2.set_title('Rezina X - reorient')
ax2.axis('off')

In [ ]:
# PRIPRAVA MATRIK
# Ta cell pripravi matriko slik in matriko razredov. Sama obdelava in priprava 
#je v okolju Google Colab zelo časovno potratna, zato je bil za del narejen na strežniku Oreh.
from tqdm import tqdm
#from google.colab import drive
#traindata ='/content/drive/My Drive/Colab Notebooks/t1-images-nove'
#drive.mount(traindata)
traindata = "./ixi_data/t1-images-rsori32"
td = os.listdir(traindata)
print('V mapi {:s} je {:d} podmap.'.format(traindata, len(td)))
#tsd = os.listdir(testdata)
#print('V mapi {:s} je {:d} podmap.'.format(testdata, len(tsd)))
X = []
train_karray = []
images = []
y  = []
Y = []
n = 0
for i in tqdm(orientacije):
    for files in os.listdir(traindata + '/' + i):
        if files.endswith("nii.gz"): 
            img = itk.ReadImage(join(traindata + '/' + i, files))
            images.append(img)
            #print(n)
            y.append(tabela_orientacij[orientacije.index(i)])
            n = n+1
print('Prva tocka')     #za spremljanje    
#print(len(images))
t1_array = np.stack([np.squeeze(itk.GetArrayFromImage(data)) for data in images])

#dodajanje 5-e dimenzije za channel, že prej preverim da je tf channels last.
train_karray = t1_array[:, :, :, :, np.newaxis]
print('Druga tocka')    #za spremljanje in lažje debuggiranje
channel_axis = -1
X = train_karray

#normalizacija
m = np.max(X)
mi = np.min(X)
X = (X - mi) / (m - mi)

#X = np.asarray(X)
print('Velikost preoblikovanega TRAIN polja: {}'.format(X.shape))
Y = np.asarray(y)
print('Velikost preoblikovanega label: {}'.format(Y.shape))

#shranim obe matriki za uvoz v google colab
np.save('X', X)
np.save('Y', Y)